In [1]:
import numpy as np
import tensorflow as tf

2024-02-03 00:48:32.445469: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-03 00:48:32.477632: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-03 00:48:32.477663: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-03 00:48:32.478371: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-03 00:48:32.483056: I tensorflow/core/platform/cpu_feature_guar

## For 196_25_10 DNN Model

In [181]:
layers = [196,25, 10]

inputs = tf.keras.layers.Input(shape=(layers[0],))
out = tf.keras.layers.Dense(layers[1], activation = 'relu')(inputs)
out = tf.keras.layers.Dense(layers[2])(out)

model = tf.keras.Model(inputs, out)

In [182]:
model_path = "../../models/"
arch_folder = "input-dense-dense/"
model_name = "196_25_10.h5"
model_in_path = model_path+arch_folder+model_name

model.load_weights(model_in_path)
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 196)]             0         
                                                                 
 dense_6 (Dense)             (None, 25)                4925      
                                                                 
 dense_7 (Dense)             (None, 10)                260       
                                                                 
Total params: 5185 (20.25 KB)
Trainable params: 5185 (20.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#### Test Accuracy

In [183]:
# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize and flatten the images
train_images_tf = train_images.reshape((-1, 28*28)) / 255.0
test_images_tf = test_images.reshape((-1, 28*28)) / 255.0

# Resize for 14 * 14 images
train_images_tf_reshaped = tf.reshape(train_images_tf, [-1, 28, 28, 1])  # Reshape to [num_samples, height, width, channels]
test_images_tf_reshaped = tf.reshape(test_images_tf, [-1, 28, 28, 1])

# Downsample images
train_images_tf_downsampled = tf.image.resize(train_images_tf_reshaped, [14, 14], method='bilinear')
test_images_tf_downsampled = tf.image.resize(test_images_tf_reshaped, [14, 14], method='bilinear')

# Flatten the images back to [num_samples, 14*14]
train_images_tf_downsampled = tf.reshape(train_images_tf_downsampled, [-1, 14*14])
test_images_tf_downsampled = tf.reshape(test_images_tf_downsampled, [-1, 14*14])

In [184]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images_tf_downsampled, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

313/313 - 0s - loss: 0.8117 - accuracy: 0.9541 - 334ms/epoch - 1ms/step

Test accuracy: 0.9541000127792358


In [185]:
def get_predictions_tf(model, test_images, batch_size=256):
    predictions = []
    for i in range(0, len(test_images), batch_size):
        batch = test_images[i:i+batch_size]
        pred = model.predict(batch)
        predictions.extend(np.argmax(pred, axis=1))
    return predictions

predictions_tf = get_predictions_tf(model, test_images_tf_downsampled)

1/1 [==============================] - 0s 30ms/step


### Generate Input Json

In [7]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617

In [8]:
def transfer_weights(layers, model, scalar = 36):
    weights = []
    biases = []
    for ind in range(len(layers)-1):
        w = [[int(model.weights[ind * 2].numpy()[i][j]*10**scalar) for j in range(layers[ind+1])] for i in range(layers[ind])]
        b = [int(model.weights[ind * 2 + 1].numpy()[i]*10**(scalar * 2)) for i in range(layers[ind+1])]
        #b = [0 for i in range(layers[ind+1])]
        weights.append(w)
        biases.append(b)

    return weights, biases


In [9]:
weights, biases = transfer_weights(layers, model, 36)

In [73]:
def relu_mod(x):
    return x if x < p // 2 else 0

def DenseInt(nInputs, nOutputs, n, input, weights, bias):
    #print (len(input), nInputs)
    
    Input = [str(input[i] % p) for i in range(nInputs)]
    Weights = [[str(weights[i][j] % p) for j in range(nOutputs)] for i in range(nInputs)]
    Bias = [str(bias[i] % p) for i in range(nOutputs)]
    
    out = [0 for _ in range(nOutputs)]
    remainder = [None for _ in range(nOutputs)]
    
    for j in range(nOutputs):
        for i in range(nInputs):
            out[j] += input[i] * weights[i][j]
        out[j] += bias[j]

        remainder[j] = str(out[j] % n)
        out[j] = out[j] // n % p
        
    return Input, Weights, Bias, out, remainder

In [127]:
def prepare_input_json(layers, weights, biases, x_in, scalar=36, relu = False):
    relu_outs = []
    dense_weights = []
    dense_biases = []
    dense_outs = []
    dense_remainders = []
    x_ins = []

    out = x_in
    for ind in range(len(weights)):
        nInputs = layers[ind]
        nOutputs = layers[ind + 1]
        #print (nInputs, nOutputs)
        x_in, w, b, out, rem = DenseInt(nInputs, nOutputs, 10 ** scalar, 
                                     out, weights[ind], biases[ind])
        
        dense_outs.append(out)
        if relu:
            out = [x if x < p//2 else 0 for x in out]
            relu_outs.append([str(x) if x !=0 else 0 for x in out ])

        #print (out)
        dense_weights.append(w)
        dense_biases.append(b)
        
        dense_remainders.append(rem)
        x_ins.append(x_in)

    
    dense_outs = [[str(x) for x in sub] for sub in dense_outs]
        
    return x_ins[0], dense_weights, dense_biases, dense_outs, dense_remainders, relu_outs, np.argmax(out)

In [143]:
X = test_images_tf_downsampled[0:1]
#X = np.random.rand(1,layers[0])


X_in = [int(x*1e36) for x in X[0]]

In [129]:
x_in, dense_weights, dense_biases, dense_outs, dense_remainders, relu_outs, pred = prepare_input_json(layers, weights, biases, X_in, scalar=36, relu=True)
#np.argmax(dense_outs[-1]), relu_outs

In [130]:
import json

in_json = {
    "in": x_in,
    "Dense32weights": dense_weights[0],
    "Dense32bias": dense_biases[0],
    "Dense32out": dense_outs[0],
    "Dense32remainder": dense_remainders[0],
    "ReLUout": relu_outs[0], 
    "Dense21weights": dense_weights[1],
    "Dense21bias": dense_biases[1],
    "Dense21out": dense_outs[1],
    "Dense21remainder": dense_remainders[1]
}


with open("./tmp/input.json", "w") as f:
    json.dump(in_json, f)

In [131]:
model.predict(X), np.argmax(model.predict(X)), pred

1/1 [==============================] - 0s 18ms/step


(array([[-2.455759 , -1.7655313, -0.4758181, -4.186792 , -1.8733556,
         -3.0348682,  6.7704954, -3.8508115, -5.1690784, -1.8425105]],
       dtype=float32),
 6,
 6)

### Phase 1 (General)

In [94]:
import subprocess, os

In [96]:
# Setup
digit = 16 # 2**15 - 18075*2 = -3000 
tmp_folder = './tmp/'
ceremony_folder = tmp_folder + 'ceremony/'
os.makedirs(ceremony_folder, exist_ok=True)
ptau_1 = ceremony_folder + 'pot12_0000.ptau'

command = ['snarkjs', 'powersoftau', 'new', 'bn128', str(digit), ptau_1,'-v']
subprocess.run(command)

[DEBUG] snarkJS: tauG1: 100000
[DEBUG] snarkJS: Calculating First Challenge Hash
[DEBUG] snarkJS: Calculate Initial Hash: tauG1
[DEBUG] snarkJS: Calculate Initial Hash: tauG2
[DEBUG] snarkJS: Calculate Initial Hash: alphaTauG1
[DEBUG] snarkJS: Calculate Initial Hash: betaTauG1
[DEBUG] snarkJS: Blank Contribution Hash:
		786a02f7 42015903 c6c6fd85 2552d272
		912f4740 e1584761 8a86e217 f71f5419
		d25e1031 afee5853 13896444 934eb04b
		903a685b 1448b755 d56f701a fe9be2ce
[INFO]  snarkJS: First Contribution Hash:
		e27d7e51 abd16bb3 c46609c7 5e963b5f
		dbe25b00 c0b93a4c 528b569a d4b50fda
		d9926ff2 781f4bae fac213db 214f30af
		e681f7be 5c1f973c c567c817 e871f958


CompletedProcess(args=['snarkjs', 'powersoftau', 'new', 'bn128', '16', './tmp/ceremony/pot12_0000.ptau', '-v'], returncode=0)

Run this in terminal as asking random input for key gen
```bash
snarkjs powersoftau contribute ./tmp/ceremony/pot12_0000.ptau ./tmp/ceremony/pot12_0001.ptau --name="1st" -v
```

In [97]:
ptau_2 = ceremony_folder + 'pot12_0001.ptau'
ptau_3 = ceremony_folder + 'pot12_final.ptau'

command = ['snarkjs', 'powersoftau', 'prepare', 'phase2', ptau_2,ptau_3, '-v']
subprocess.run(command)

[DEBUG] snarkJS: Starting section: tauG1
[DEBUG] snarkJS: tauG1: fft 0 mix start: 0/1
[DEBUG] snarkJS: tauG1: fft 0 mix end: 0/1
[DEBUG] snarkJS: tauG1: fft 1 mix start: 0/1
[DEBUG] snarkJS: tauG1: fft 1 mix end: 0/1
[DEBUG] snarkJS: tauG1: fft 2 mix start: 0/1
[DEBUG] snarkJS: tauG1: fft 2 mix end: 0/1
[DEBUG] snarkJS: tauG1: fft 3 mix start: 0/1
[DEBUG] snarkJS: tauG1: fft 3 mix end: 0/1
[DEBUG] snarkJS: tauG1: fft 4 mix start: 0/2
[DEBUG] snarkJS: tauG1: fft 4 mix start: 1/2
[DEBUG] snarkJS: tauG1: fft 4 mix end: 0/2
[DEBUG] snarkJS: tauG1: fft 4 mix end: 1/2
[DEBUG] snarkJS: tauG1: fft  4  join: 4/4
[DEBUG] snarkJS: tauG1: fft 4 join  4/4  1/1 0/1
[DEBUG] snarkJS: tauG1: fft 5 mix start: 0/4
[DEBUG] snarkJS: tauG1: fft 5 mix start: 1/4
[DEBUG] snarkJS: tauG1: fft 5 mix start: 2/4
[DEBUG] snarkJS: tauG1: fft 5 mix start: 3/4
[DEBUG] snarkJS: tauG1: fft 5 mix end: 0/4
[DEBUG] snarkJS: tauG1: fft 5 mix end: 1/4
[DEBUG] snarkJS: tauG1: fft 5 mix end: 3/4
[DEBUG] snarkJS: tauG1: fft 5 m

CompletedProcess(args=['snarkjs', 'powersoftau', 'prepare', 'phase2', './tmp/ceremony/pot12_0001.ptau', './tmp/ceremony/pot12_final.ptau', '-v'], returncode=0)

### Phase 2 (circuit-specific)

In [106]:
import os, subprocess
circuit_folder = "./golden_circuits/"
target_circom = "_".join(str(x) for x in layers) + '.circom'

command = ['circom', circuit_folder + target_circom, "--r1cs", "--wasm", "--sym", "-o", tmp_folder]
subprocess.run(command)

template instances: 15
non-linear constraints: 18075
linear constraints: 0
public inputs: 0
private inputs: 5476 (5466 belong to witness)
public outputs: 10
wires: 23442
labels: 72908
Written successfully: ./tmp/196_25_10.r1cs
Written successfully: ./tmp/196_25_10.sym
Written successfully: ./tmp/196_25_10_js/196_25_10.wasm
Everything went okay


CompletedProcess(args=['circom', './golden_circuits/196_25_10.circom', '--r1cs', '--wasm', '--sym', '-o', './tmp/'], returncode=0)

In [108]:
r1cs_path = tmp_folder + "_".join(str(x) for x in layers) + ".r1cs"
zkey_1 = ceremony_folder + 'test_0000.zkey'
command = ['snarkjs', 'groth16', 'setup', r1cs_path, ptau_3, zkey_1]
subprocess.run(command)

[INFO]  snarkJS: Reading r1cs
[INFO]  snarkJS: Reading tauG1
[INFO]  snarkJS: Reading tauG2
[INFO]  snarkJS: Reading alphatauG1
[INFO]  snarkJS: Reading betatauG1
[INFO]  snarkJS: Circuit hash: 
		0b3b490d 51a55986 9382846d f7eb35fe
		d5bab2e3 64ee4b8b e27d2a72 c1829728
		968cd1ec b3c19184 c66ed7a3 546e71ab
		6cea4959 acaebae1 598c9bc3 8c91130f


CompletedProcess(args=['snarkjs', 'groth16', 'setup', './tmp/196_25_10.r1cs', './tmp/ceremony/pot12_final.ptau', './tmp/ceremony/test_0000.zkey'], returncode=0)

```bash
snarkjs zkey contribute ./tmp/ceremony/test_0000.zkey ./tmp/ceremony/test_0001.zkey --name="usr1" -v

```

In [109]:
veri_key = ceremony_folder + 'vk.json'
command = ['snarkjs', 'zkey', 'export','verificationkey', zkey_1, veri_key]
subprocess.run(command)

CompletedProcess(args=['snarkjs', 'zkey', 'export', 'verificationkey', './tmp/ceremony/test_0000.zkey', './tmp/ceremony/vk.json'], returncode=0)

### Benchmarking Proof Gen

In [167]:
import concurrent.futures, json, threading, psutil, time
def monitor_memory(pid, freq = 0.01):
    p = psutil.Process(pid)
    max_memory = 0
    while True:
        try:
            mem = p.memory_info().rss / (1024 * 1024)
            max_memory = max(max_memory, mem)
        except psutil.NoSuchProcess:
            break  # Process has finished
        time.sleep(freq)  # Poll every second
        
    #print(f"Maximum memory used: {max_memory} MB")
    return max_memory

def execute_and_monitor(command):
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.submit(monitor_memory, process.pid)
        stdout, stderr = process.communicate()
        max_memory = future.result()
    #print(f"Maximum memory used: {max_memory} MB")
    return stdout, stderr, max_memory

In [186]:
def benchmark(test_images, predictions, weights, biases, layers, tmp_folder, input_path, zkey, veri_key):
    loss = 0

    target_circom = "_".join(str(x) for x in layers) + '.circom'

    json_folder = tmp_folder + target_circom[:-7] + "_js/"
    wit_json_file = json_folder + "generate_witness.js"
    wasm_file = json_folder + target_circom[:-7] + ".wasm"
    input_path = tmp_folder + "input.json"
    wit_file = tmp_folder + "witness.wtns"

    mem_usage = []
    time_cost = []
    benchmark_start_time = time.time()

    for i in range(len(test_images)):
        cost = 0
        X = test_images[i:i+1]
        start_time = time.time()
        X_in = [int(x*1e36) for x in X[0]]
        x_in, dense_weights, dense_biases, dense_outs, dense_remainders, relu_outs, pred = prepare_input_json(layers, weights, biases, X_in, scalar=36, relu=True)

        in_json = {
            "in": x_in,
            "Dense32weights": dense_weights[0],
            "Dense32bias": dense_biases[0],
            "Dense32out": dense_outs[0],
            "Dense32remainder": dense_remainders[0],
            "ReLUout": relu_outs[0], 
            "Dense21weights": dense_weights[1],
            "Dense21bias": dense_biases[1],
            "Dense21out": dense_outs[1],
            "Dense21remainder": dense_remainders[1]
        }

        with open(input_path, "w") as f:
            json.dump(in_json, f)

        if pred != predictions[i]:
            loss += 1
            print ("Loss happens on index", i)

        commands = [['node', wit_json_file, wasm_file, input_path, wit_file],
                    ['snarkjs', 'groth16', 'prove',zkey, wit_file, tmp_folder+'proof.json', tmp_folder+'public.json']]
                    #['snarkjs', 'groth16', 'verify',veri_key, tmp_folder+'public.json', tmp_folder+'proof.json']]

        for command in commands:
            _, _, usage = execute_and_monitor(command)
            cost += usage
        #print ("stdout:", stdout)
            
        mem_usage.append(cost)
        time_cost.append(time.time() - start_time)
    
    print ("Total time:", time.time() - benchmark_start_time)
    return loss, mem_usage, time_cost


In [171]:
test_size = 2500
loss, mem_usage, time_cost = benchmark(test_images_tf[:test_size], predictions_tf[:test_size], weights, biases, layers, "./tmp/", "./tmp/input.json", zkey_1, veri_key)

Total time: 2924.856101989746


In [173]:
import pandas as pd

csv_path = '../../benchmarks/benchmark_results.csv'

columns = ['Framework', 'Architecture', '# Layers', '# Parameters', 'Testing Size', 'Accuracy Loss (%)', 
           'Avg Memory Usage (MB)', 'Std Memory Usage', 'Avg Proving Time (s)', 'Std Proving Time']

# Check if the CSV file exists
if not os.path.isfile(csv_path):
    # Create a DataFrame with the specified columns
    df = pd.DataFrame(columns=columns)
    # Save the DataFrame as a CSV file
    df.to_csv(csv_path, index=False)
else:
    print(f"File '{csv_path}' already exists.")

df = pd.read_csv(csv_path)

File '../../benchmarks/benchmark_results.csv' already exists.


In [174]:
new_row = {
    'Framework': ['circomlib-ml (tensorflow)'],
    'Architecture': ['Input-Dense-Dense (196 * 25 * 10'],
    '# Layers': [3],
    '# Parameters': [5185],
    'Testing Size': [test_size],
    'Accuracy Loss (%)': [loss/test_size * 100],
    'Avg Memory Usage (MB)': [sum(mem_usage) / len(mem_usage)],
    'Std Memory Usage': [pd.Series(mem_usage).std()],
    'Avg Proving Time (s)': [sum(time_cost) / len(time_cost)],
    'Std Proving Time': [pd.Series(time_cost).std()]
}

new_row_df = pd.DataFrame(new_row)


In [175]:
df = pd.concat([df, new_row_df], ignore_index=True)
df.to_csv(csv_path, index=False)

In [176]:
df

,Framework,Architecture,# Layers,# Parameters,Testing Size,Accuracy Loss (%),Avg Memory Usage (MB),Std Memory Usage,Avg Proving Time (s),Std Proving Time
0,opml (pytorch),Input-Dense-Dense (784 * 56 * 10),3,44543,250,0.40,88.998094,2.285579,3.655122,0.440126
1,opml (pytorch),"Input-Dense-Dense (784 * 56 * 10, w/ relu)",3,44543,1000,20.90,89.122078,2.247846,3.664727,0.433921
2,opml (pytorch),Input-Dense-Dense (784 * 56 * 10),3,44543,2500,0.72,89.120883,2.254392,3.609974,0.421732
3,opml (pytorch),Input-Dense-Dense (196 * 25 * 10,3,5185,5000,3.52,74.351948,1.419725,0.809440,0.074357
4,circomlib-ml (tensorflow),Input-Dense-Dense (196 * 25 * 10,3,5185,2500,0.00,998.735128,8.040170,1.169482,0.108491


#### Test

In [132]:
# Gen Witness
json_folder = tmp_folder + target_circom[:-7] + "_js/"
wit_json_file = json_folder + "generate_witness.js"
wasm_file = json_folder + target_circom[:-7] + ".wasm"
input_path = tmp_folder + "input.json"
wit_file = tmp_folder + "witness.wtns"

command = ['node', wit_json_file, wasm_file, input_path, wit_file]
subprocess.run(command)

CompletedProcess(args=['node', './tmp/196_25_10_js/generate_witness.js', './tmp/196_25_10_js/196_25_10.wasm', './tmp/input.json', './tmp/witness.wtns'], returncode=0)

In [133]:
command = ['snarkjs', 'groth16', 'prove',zkey_1, wit_file, tmp_folder+'proof.json', tmp_folder+'public.json', ]
subprocess.run(command)

CompletedProcess(args=['snarkjs', 'groth16', 'prove', './tmp/ceremony/test_0000.zkey', './tmp/witness.wtns', './tmp/proof.json', './tmp/public.json'], returncode=0)

In [134]:
command = ['snarkjs', 'groth16', 'verify',veri_key, tmp_folder+'public.json', tmp_folder+'proof.json']
subprocess.run(command)

[INFO]  snarkJS: OK!


CompletedProcess(args=['snarkjs', 'groth16', 'verify', './tmp/ceremony/vk.json', './tmp/public.json', './tmp/proof.json'], returncode=0)

In [177]:
## For 784_56_10 DNN Model

In [187]:
layers = [784, 56, 10]

inputs = tf.keras.layers.Input(shape=(layers[0],))
out = tf.keras.layers.Dense(layers[1], activation = 'relu')(inputs)
out = tf.keras.layers.Dense(layers[2])(out)

model = tf.keras.Model(inputs, out)

In [188]:
model_path = "../../models/"
arch_folder = "input-dense-dense/"
model_name = "784_56_10.h5"
model_in_path = model_path+arch_folder+model_name

model.load_weights(model_in_path)
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 784)]             0         
                                                                 
 dense_8 (Dense)             (None, 56)                43960     
                                                                 
 dense_9 (Dense)             (None, 10)                570       
                                                                 
Total params: 44530 (173.95 KB)
Trainable params: 44530 (173.95 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [189]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images_tf, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

313/313 - 0s - loss: 0.5807 - accuracy: 0.9740 - 342ms/epoch - 1ms/step

Test accuracy: 0.9739999771118164


In [191]:
def get_predictions_tf(model, test_images, batch_size=256):
    predictions = []
    for i in range(0, len(test_images), batch_size):
        batch = test_images[i:i+batch_size]
        pred = model.predict(batch)
        predictions.extend(np.argmax(pred, axis=1))
    return predictions

predictions_tf = get_predictions_tf(model, test_images_tf)

1/1 [==============================] - 0s 19ms/step


### Generate Input Json with Sampled Img

In [192]:
weights, biases = transfer_weights(layers, model, 36)

In [194]:
X = test_images_tf[0:1]

X_in = [int(x*1e36) for x in X[0]]

In [195]:
x_in, dense_weights, dense_biases, dense_outs, dense_remainders, relu_outs, pred = prepare_input_json(layers, weights, biases, X_in, scalar=36, relu=True)


In [196]:
import json

in_json = {
    "in": x_in,
    "Dense32weights": dense_weights[0],
    "Dense32bias": dense_biases[0],
    "Dense32out": dense_outs[0],
    "Dense32remainder": dense_remainders[0],
    "ReLUout": relu_outs[0], 
    "Dense21weights": dense_weights[1],
    "Dense21bias": dense_biases[1],
    "Dense21out": dense_outs[1],
    "Dense21remainder": dense_remainders[1]
}


with open("./tmp/input.json", "w") as f:
    json.dump(in_json, f)

In [197]:
model.predict(X), np.argmax(model.predict(X)), pred

1/1 [==============================] - 0s 14ms/step


(array([[ -5.5879316,  -8.324502 ,  -2.2807932,   2.9312103, -11.059059 ,
          -4.010176 , -20.9365   ,   7.937256 ,  -0.9530401,  -1.6678814]],
       dtype=float32),
 7,
 7)

### Trusted Setup

In [ ]:
# Setup
digit = 17 # 2**15 - 18075*2 = -3000 
tmp_folder = './tmp/'
ceremony_folder = tmp_folder + 'big_ceremony/'
os.makedirs(ceremony_folder, exist_ok=True)
ptau_1 = ceremony_folder + 'pot12_0000.ptau'

command = ['snarkjs', 'powersoftau', 'new', 'bn128', str(digit), ptau_1,'-v']
subprocess.run(command)